# Unit 12 - Tales from the Crypto

---


## 1. Sentiment Analysis

Use the [newsapi](https://newsapi.org/) to pull the latest news articles for Bitcoin and Ethereum and create a DataFrame of sentiment scores for each coin.

Use descriptive statistics to answer the following questions:
1. Which coin had the highest mean positive score?
2. Which coin had the highest negative score?
3. Which coin had the highest positive score?

In [327]:
# Initial imports
import os
import pandas as pd
from dotenv import load_dotenv
from newsapi import NewsApiClient
import nltk as nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import operator

%matplotlib inline

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/akanksha/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [328]:
# Read your api key environment variable
api_key = os.getenv("NEWS_API_KEY")

In [329]:
# Create a newsapi client
newsapi = NewsApiClient(api_key=api_key)

In [330]:
# Fetch the Bitcoin news articles
bitcoin_articles = newsapi.get_everything(q="bitcoin", language="en")

In [331]:
# Fetch the Ethereum news articles
ethereum_articles = newsapi.get_everything(q="ethereum", language="en")

In [332]:
# Method to get sentiments for articles 
def get_sentiments(articles):
    """
    Gets the sentiments for all the articles provided
    Params:
        articles- dictionary of the response from newspi call
    Returns:
        sentiments_df- Dataframe of all the titles and associated sentiments
    """
    sentiments = []
    for article in articles['articles']:
        sentiment = analyzer.polarity_scores(article['content'])
        sentiments.append({
            'title': article['title'], 
            'compound': sentiment["compound"],
            'pos': sentiment["pos"],
            'neu': sentiment["neu"],
            'neg': sentiment["neg"]
            })
    sentiments_df = pd.DataFrame(sentiments)
    return sentiments_df

In [333]:
# Create the Bitcoin sentiment scores DataFrame
bitcoin_sentiment_df = get_sentiments(bitcoin_articles)
bitcoin_sentiment_df.head()

,title,compound,pos,neu,neg
0,New York passes a bill to limit bitcoin mining,-0.5574,0.000,0.902,0.098
1,Miami’s Bitcoin Conference Left a Trail of Har...,0.0772,0.036,0.964,0.000
2,Why the Central African Republic adopted Bitcoin,0.5106,0.164,0.836,0.000
3,Chipotle now accepts cryptocurrency payments,-0.1027,0.000,0.960,0.040
4,Bitcoin value drops by 50% since November peak,0.3400,0.072,0.928,0.000


In [334]:
# Create the Ethereum sentiment scores DataFrame
ethereum_sentiment_df = get_sentiments(ethereum_articles)
ethereum_sentiment_df.head()

,title,compound,pos,neu,neg
0,Here's what NFTs look like on Instagram,0.6486,0.135,0.865,0.000
1,GameStop Dunks Its Head Into the Crypto Kiddie...,-0.1027,0.000,0.960,0.040
2,PayPal Lets Users Transfer Bitcoin and Ethereu...,0.3182,0.059,0.941,0.000
3,GameStop Launching Digital Wallet for Cryptocu...,0.1280,0.046,0.954,0.000
4,New York passes a bill to limit bitcoin mining,-0.5574,0.000,0.902,0.098


In [335]:
# Describe the Bitcoin Sentiment
bitcoin_sentiment_describe = bitcoin_sentiment_df.describe()
bitcoin_sentiment_describe

,compound,pos,neu,neg
count,20.000000,20.000000,20.000000,20.000000
mean,-0.093710,0.048600,0.883100,0.068400
std,0.394537,0.050315,0.075115,0.067337
min,-0.859300,0.000000,0.646000,0.000000
25%,-0.386825,0.000000,0.850750,0.030000
50%,-0.051350,0.041500,0.888500,0.063000
75%,0.249275,0.077500,0.924250,0.087500
max,0.510600,0.164000,1.000000,0.300000


In [336]:
# Describe the Ethereum Sentiment
ethereum_sentiment_describe = ethereum_sentiment_df.describe()
ethereum_sentiment_describe

,compound,pos,neu,neg
count,20.000000,20.000000,20.000000,20.000000
mean,-0.008160,0.041350,0.918200,0.040550
std,0.398847,0.051657,0.052626,0.049995
min,-0.690800,0.000000,0.822000,0.000000
25%,-0.381800,0.000000,0.891250,0.000000
50%,0.000000,0.017000,0.924000,0.020000
75%,0.323650,0.072750,0.955000,0.077000
max,0.690800,0.178000,1.000000,0.178000


### Questions:

Q: Which coin had the highest mean positive score?

A: Bitcoin has the highest mean positive score of 0.061750000000000006

In [337]:
print(f'Bitcoin mean positive score: {bitcoin_sentiment_describe.loc["mean"]["pos"]}')
print(f'Ethereum mean positive score: {ethereum_sentiment_describe.loc["mean"]["pos"]}')


Bitcoin mean positive score: 0.048600000000000004
Ethereum mean positive score: 0.04134999999999999


Q: Which coin had the highest compound score?

A: Bitcoin had the highest compound score of 0.7506

In [338]:
print(f'Bitcoin max compound score: {bitcoin_sentiment_describe.loc["max"]["compound"]}')
print(f'Ethereum max compound score: {ethereum_sentiment_describe.loc["max"]["compound"]}')

Bitcoin max compound score: 0.5106
Ethereum max compound score: 0.6908


Q: Which coin had the highest positive score?

A: Bitcoin had the highest positive score of 0.202

In [339]:
print(f'Bitcoin max positive score: {bitcoin_sentiment_describe.loc["max"]["pos"]}')
print(f'Ethereum max positive score: {ethereum_sentiment_describe.loc["max"]["pos"]}')

Bitcoin max positive score: 0.164
Ethereum max positive score: 0.178


---

## 2. Natural Language Processing
---
###   Tokenizer

In this section, you will use NLTK and Python to tokenize the text for each coin. Be sure to:
1. Lowercase each word.
2. Remove Punctuation.
3. Remove Stopwords.

In [340]:
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
from string import punctuation
import re

In [341]:
# Instantiate the lemmatizer
lemmatizer = WordNetLemmatizer()

# Create a list of stopwords
stop_words = stopwords.words('english')

# Expand the default stopwords list if necessary
stop_words.append('ha')

In [342]:
# Complete the tokenizer function
def tokenizer(text, lemmatizer, stop_words):
    """Tokenizes text."""
    
    # Remove the punctuation from text
    regex = re.compile("[^a-zA-Z ]")
    re_clean_text = regex.sub('', text)

    # Create a tokenized list of the words
    words = word_tokenize(re_clean_text)
    
    # Lemmatize words into root words
    lem = [lemmatizer.lemmatize(word) for word in words]
   
    # Convert the words to lowercase and remove the stop words
    tokens = [word.lower() for word in lem if word.lower() not in stop_words]
    
    return tokens

In [343]:
# Create a new tokens column for Bitcoin
stop_words_bitcoin = stop_words
stop_words_bitcoin.append('ethereum')

bitcoin_sentiment_df['tokens'] = [tokenizer(article['content'],lemmatizer,stop_words_bitcoin) for article in bitcoin_articles['articles']]
bitcoin_sentiment_df.head()

,title,compound,pos,neu,neg,tokens
0,New York passes a bill to limit bitcoin mining,-0.5574,0.000,0.902,0.098,"[new, york, lawmaker, passed, bill, would, tem..."
1,Miami’s Bitcoin Conference Left a Trail of Har...,0.0772,0.036,0.964,0.000,"[even, though, number, womenfocused, crypto, s..."
2,Why the Central African Republic adopted Bitcoin,0.5106,0.164,0.836,0.000,"[image, caption, president, faustinarchange, t..."
3,Chipotle now accepts cryptocurrency payments,-0.1027,0.000,0.960,0.040,"[reportedly, pay, burrito, taco, bitcoin, digi..."
4,Bitcoin value drops by 50% since November peak,0.3400,0.072,0.928,0.000,"[image, source, getty, imagesthe, value, bitco..."


In [344]:
# Create a new tokens column for Ethereum
stop_words_ethereum = stop_words
stop_words_ethereum.append('bitcoin')
ethereum_sentiment_df['tokens'] = [tokenizer(article['content'],lemmatizer,stop_words_ethereum) for article in ethereum_articles['articles']]
ethereum_sentiment_df.head()

,title,compound,pos,neu,neg,tokens
0,Here's what NFTs look like on Instagram,0.6486,0.135,0.865,0.000,"[meta, revealed, nfts, work, instagram, usbase..."
1,GameStop Dunks Its Head Into the Crypto Kiddie...,-0.1027,0.000,0.960,0.040,"[gamestop, officially, thrown, headlong, web, ..."
2,PayPal Lets Users Transfer Bitcoin and Ethereu...,0.3182,0.059,0.941,0.000,"[ability, conduct, external, transfer, paypals..."
3,GameStop Launching Digital Wallet for Cryptocu...,0.1280,0.046,0.954,0.000,"[gamestop, going, allin, crypto, video, game, ..."
4,New York passes a bill to limit bitcoin mining,-0.5574,0.000,0.902,0.098,"[new, york, lawmaker, passed, bill, would, tem..."


---

### NGrams and Frequency Analysis

In this section you will look at the ngrams and word frequency for each coin. 

1. Use NLTK to produce the n-grams for N = 2. 
2. List the top 10 words for each coin. 

In [345]:
from collections import Counter
from nltk import ngrams

In [346]:
def get_tokens(tokens_dataframe):
    tokens = []
    for i in range(tokens_dataframe.shape[0]):
        tokens += tokens_dataframe.iloc[i]['tokens']
    return tokens

In [347]:
# Generate the N-grams for given dataframe
def get_ngrams(tokens, n=2):
    """
    Generates the N-grams for given dataframe
    Params:
        tokens- list to generate n-grams from
        n- the degree of the ngrams
    Returns:
        ngrams_dict- dictionary of n-grams, default=2
    """
    ngrams_count = Counter(ngrams(tokens, n=2))
    ngrams_dict = dict(ngrams_count)
    ngrams_dict = dict(sorted(ngrams_dict.items(), key=operator.itemgetter(1),reverse=True))
    return ngrams_dict

In [348]:
# Generate the Bitcoin N-grams where N=2
bitcoin_tokens = get_tokens(bitcoin_sentiment_df)
bitcoin_ngrams = get_ngrams(bitcoin_tokens)
bitcoin_ngrams

{('may', 'reuters'): 4,
 ('reuters', 'bitcoin'): 4,
 ('world', 'biggest'): 3,
 ('biggest', 'bestknown'): 3,
 ('bestknown', 'cryptocurrency'): 3,
 ('char', 'may'): 3,
 ('new', 'blockchain'): 3,
 ('seth', 'greens'): 3,
 ('b', 'char'): 2,
 ('char', 'image'): 2,
 ('pay', 'burrito'): 2,
 ('losing', 'previous'): 2,
 ('previous', 'closebitcoin'): 2,
 ('closebitcoin', 'world'): 2,
 ('gmt', 'monday'): 2,
 ('interest', 'rate'): 2,
 ('char', 'terra'): 2,
 ('blockchain', 'new'): 2,
 ('new', 'luna'): 2,
 ('luna', 'token'): 2,
 ('greens', 'animated'): 2,
 ('animated', 'show'): 2,
 ('show', 'starring'): 2,
 ('starring', 'stolen'): 2,
 ('stolen', 'nft'): 2,
 ('nft', 'air'): 2,
 ('air', 'largest'): 2,
 ('largest', 'stablecoin'): 2,
 ('stablecoin', 'issuer'): 2,
 ('issuer', 'world'): 2,
 ('world', 'launched'): 2,
 ('launched', 'token'): 2,
 ('new', 'york'): 1,
 ('york', 'lawmaker'): 1,
 ('lawmaker', 'passed'): 1,
 ('passed', 'bill'): 1,
 ('bill', 'would'): 1,
 ('would', 'temporarily'): 1,
 ('temporarily

In [359]:
# Generate the Ethereum N-grams where N=2
ethereum_tokens = get_tokens(ethereum_sentiment_df)
ethereum_ngrams = get_ngrams(ethereum_tokens)
ethereum_ngrams

{('may', 'reuters'): 3,
 ('world', 'biggest'): 3,
 ('biggest', 'bestknown'): 3,
 ('bestknown', 'cryptocurrency'): 3,
 ('char', 'gamestop'): 2,
 ('lite', 'hash'): 2,
 ('hash', 'rate'): 2,
 ('graphic', 'card'): 2,
 ('char', 'may'): 2,
 ('losing', 'previous'): 2,
 ('previous', 'closebitcoin'): 2,
 ('closebitcoin', 'world'): 2,
 ('gmt', 'monday'): 2,
 ('char', 'cryptocurrency'): 2,
 ('cryptocurrency', 'boom'): 2,
 ('boom', 'past'): 2,
 ('past', 'year'): 2,
 ('year', 'helped'): 2,
 ('helped', 'propel'): 2,
 ('propel', 'newer'): 2,
 ('newer', 'market'): 2,
 ('market', 'record'): 2,
 ('record', 'height'): 2,
 ('height', 'digital'): 2,
 ('digital', 'collectible'): 2,
 ('collectible', 'known'): 2,
 ('known', 'nftsthe'): 2,
 ('nftsthe', 'craze'): 2,
 ('craze', 'pushed'): 2,
 ('pushed', 'total'): 2,
 ('total', 'nft'): 2,
 ('nft', 'sale'): 2,
 ('sale', 'volume'): 2,
 ('volume', 'billion'): 2,
 ('billion', 'ov'): 2,
 ('ov', 'char'): 2,
 ('meta', 'revealed'): 1,
 ('revealed', 'nfts'): 1,
 ('nfts', '

In [361]:
# Function token_count generates the top 10 words for a given coin
def token_count(tokens, N=3):
    """Returns the top N tokens from the frequency count"""
    return Counter(tokens).most_common(N)

In [362]:
# Use token_count to get the top 10 words for Bitcoin
bitcoin_token_count = token_count(bitcoin_tokens, 10)
bitcoin_token_count

[('char', 19),
 ('bitcoin', 11),
 ('new', 8),
 ('cryptocurrency', 8),
 ('world', 6),
 ('blockchain', 5),
 ('week', 5),
 ('reuters', 5),
 ('may', 4),
 ('biggest', 4)]

In [363]:
# Use token_count to get the top 10 words for Ethereum
ethereum_token_count = token_count(ethereum_tokens, 10)
ethereum_token_count

[('char', 20),
 ('cryptocurrency', 11),
 ('may', 5),
 ('world', 5),
 ('digital', 4),
 ('new', 4),
 ('reuters', 4),
 ('market', 4),
 ('year', 4),
 ('monday', 3)]

---

### Word Clouds

In this section, you will generate word clouds for each coin to summarize the news for each coin

In [364]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = [20.0, 10.0]

In [354]:
# Generate the Bitcoin word cloud
# YOUR CODE HERE!

In [355]:
# Generate the Ethereum word cloud
# YOUR CODE HERE!

---
## 3. Named Entity Recognition

In this section, you will build a named entity recognition model for both Bitcoin and Ethereum, then visualize the tags using SpaCy.

In [356]:
import spacy
from spacy import displacy

In [357]:
# Download the language model for SpaCy
# !python -m spacy download en_core_web_sm

In [358]:
# Load the spaCy model
nlp = spacy.load('en_core_web_sm')

OSError: [E050] Can't find model 'en_core_web_sm'. It doesn't seem to be a Python package or a valid path to a data directory.

---
### Bitcoin NER

In [ ]:
# Concatenate all of the Bitcoin text together
# YOUR CODE HERE!

In [ ]:
# Run the NER processor on all of the text
# YOUR CODE HERE!

# Add a title to the document
# YOUR CODE HERE!

In [ ]:
# Render the visualization
# YOUR CODE HERE!

In [ ]:
# List all Entities
# YOUR CODE HERE!

---

### Ethereum NER

In [ ]:
# Concatenate all of the Ethereum text together
# YOUR CODE HERE!

In [ ]:
# Run the NER processor on all of the text
# YOUR CODE HERE!

# Add a title to the document
# YOUR CODE HERE!

In [ ]:
# Render the visualization
# YOUR CODE HERE!

In [ ]:
# List all Entities
# YOUR CODE HERE!

---